In [1]:
from itertools import combinations, permutations, combinations_with_replacement
import matplotlib.pyplot as plt

In [15]:
players=[('Messi',['AD','MD','BU','MOC'],'ST'),('Griezmann',['AD','AG','MG','MD','BU','MOC'],'ST'),\
         ('Dembele',['AD','MD','AG','MG'],'ST'),('Suarez',['BU'],'ST'),\
         ('Vidal',['MOC','MC','MDC','MD'],'MID'),('Artur',['MOC','MC','MDC'],'MID'),\
         ('De Jong',['MOC','MC','MDC'],'MID'),('Busquets',['MC','MDC'],'MID'),\
         ('Rakitic',['MC','MDC'],'MID'),('Pique',['DC','DD'],'DEF'),('Umtiti',['DC','DG'],'DEF'),\
         ('Lenglet',['DC'],'DEF'),('Alba',['DG'],'DEF'),('Semedo',['DD','MD','AD'],'DEF')]
players[0]

('Messi', ['AD', 'MD', 'BU', 'MOC'], 'ST')

In [16]:
all_system=[[(4,4,2),['DG','DC','DC','DD','MG','MC','MC','MD','BU','BU']],\
         [(4,3,3),['DG','DC','DC','DD','AG','AD','MC','MC','MOC','BU']],\
         [(4,5,1),['DG','DC','DC','DD','MDC','MDC','MOC','AG','AD','BU']],\
         [(3,4,3),['DC','DC','DC','MD','AG','AD','MC','MC','MG','BU']],\
         [(4,2,4),['DG','DC','DC','DD','MC','MC','AG','AD','BU','BU']]]
all_system[0]

[(4, 4, 2), ['DG', 'DC', 'DC', 'DD', 'MG', 'MC', 'MC', 'MD', 'BU', 'BU']]

### 1. Méthode simple: attaquant/milieu/défenseur

In [4]:
def position_group(players): # give the list of the strikers, defenders and midfielders
    l=[[],[],[]]
    for i in range(len(players)):
        if players[i][2]=='ST':
            l[2].append(players[i][0])
        elif players[i][2]=='DEF':
            l[0].append(players[i][0])
        else:
            l[1].append(players[i][0])
    return l

In [5]:
position_group(players)

[['Pique', 'Umtiti', 'Lenglet', 'Alba', 'Semedo'],
 ['Vidal', 'Artur', 'De Jong', 'Busquets', 'Rakitic'],
 ['Messi', 'Griezmann', 'Dembele', 'Suarez']]

In [6]:
def line_up_by_system(players,system):#all the lines up by system
    n_defender=system[0][0]
    n_midfielder=system[0][1]
    n_striker=system[0][2]
    list_defender=list(combinations(position_group(players)[0],n_defender))
    list_midfielder=list(combinations(position_group(players)[1],n_midfielder))
    list_striker=list(combinations(position_group(players)[2],n_striker))
    #We have the list of the combinations possible in defense, in attack and in the middle. 
    #We have now to combine them to have a complete line_up
    total=list_defender+list_midfielder+list_striker
    global_list=list(combinations(total,3))
    n=len(global_list)
    #Keep only the good elements (not to have 4 def+4 def +2 Att in a 4-4-2 system)
    final=[]
    for i in range(n):
        if global_list[i][0] in list_defender:
            if global_list[i][1] in list_midfielder:
                if global_list[i][2] in list_striker:
                    final.append(global_list[i])
    return final

### 2. Méthode avancée: chaque joueur a plusieurs postes

On aura ainsi les compositions plus proches de la réalité. Par exemple on n'aura pas une défense à 3 avec deux DG et un DD qui ne jouent jamais à ce poste.

In [9]:
def position_player(players,system): #return a list of possible players for each position in a system
    position_player=[[],[],[],[],[],[],[],[],[],[]]
    n=len(players)
    for i in range(len(system[1])):
        for j in range(n):
            if system[1][i] in players[j][1]:
                position_player[i].append(players[j][0])
    return position_player, system[1]

In [10]:
position_player(players,all_system[0])

([['Umtiti', 'Alba'],
  ['Pique', 'Umtiti', 'Lenglet'],
  ['Pique', 'Umtiti', 'Lenglet'],
  ['Pique', 'Semedo'],
  ['Griezmann', 'Dembele'],
  ['Vidal', 'Artur', 'De Jong', 'Busquets', 'Rakitic'],
  ['Vidal', 'Artur', 'De Jong', 'Busquets', 'Rakitic'],
  ['Messi', 'Griezmann', 'Dembele', 'Vidal', 'Semedo'],
  ['Messi', 'Griezmann', 'Suarez'],
  ['Messi', 'Griezmann', 'Suarez']],
 ['DG', 'DC', 'DC', 'DD', 'MG', 'MC', 'MC', 'MD', 'BU', 'BU'])

In [18]:
#Non-abouti
def line_up_by_system_bis(players,system):#all the lines up by system
    n_defender=system[0][0]
    n_midfielder=system[0][1]
    n_striker=system[0][2]
    total=[]
    for i in range(10):
        list_by_pos=[list(permutations(position_player(players,system)[0][i],1))]
        total=total+list_by_pos
    global_list=[]
    for i in range(10):
        global_list=global_list+list(permutations(total[i],len(total[i])))
    return total, global_list

In [19]:
line_up_by_system_bis(players,all_system[2]) #pb de combinaison ici

([[('Umtiti',), ('Alba',)],
  [('Pique',), ('Umtiti',), ('Lenglet',)],
  [('Pique',), ('Umtiti',), ('Lenglet',)],
  [('Pique',), ('Semedo',)],
  [('Vidal',), ('Artur',), ('De Jong',), ('Busquets',), ('Rakitic',)],
  [('Vidal',), ('Artur',), ('De Jong',), ('Busquets',), ('Rakitic',)],
  [('Messi',), ('Griezmann',), ('Vidal',), ('Artur',), ('De Jong',)],
  [('Griezmann',), ('Dembele',)],
  [('Messi',), ('Griezmann',), ('Dembele',), ('Semedo',)],
  [('Messi',), ('Griezmann',), ('Suarez',)]],
 [(('Umtiti',), ('Alba',)),
  (('Alba',), ('Umtiti',)),
  (('Pique',), ('Umtiti',), ('Lenglet',)),
  (('Pique',), ('Lenglet',), ('Umtiti',)),
  (('Umtiti',), ('Pique',), ('Lenglet',)),
  (('Umtiti',), ('Lenglet',), ('Pique',)),
  (('Lenglet',), ('Pique',), ('Umtiti',)),
  (('Lenglet',), ('Umtiti',), ('Pique',)),
  (('Pique',), ('Umtiti',), ('Lenglet',)),
  (('Pique',), ('Lenglet',), ('Umtiti',)),
  (('Umtiti',), ('Pique',), ('Lenglet',)),
  (('Umtiti',), ('Lenglet',), ('Pique',)),
  (('Lenglet',), ('P